### Import Required Libraries and Set Up Environment Variables

In [4]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [5]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [6]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "(headline,headline,web_url,snippet,source,keywords,pub_date,byline,word_count,word_count)"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
nyt_movie_query = f"{url}api-key={nyt_api_key}&fq={filter_query}&fl={field_list}&sort={sort}&begin_date={begin_date}&end_date={end_date}"

#
#print(nyt_movie_query)

#with open('test_output.txt', 'w') as f:
#        f.write(nyt_movie_query)
#        f.write('\n')


In [7]:
# Create an empty list to store the reviews
nyt_movies_list = []


# loop through pages 0-19

    # create query with a page number
    # API results show 10 articles at a time
for page in range(0, 20):
    query_url = f"{nyt_movie_query}&page={page}"
    #print(query_url)
        
# Make a "GET" request and retrieve the JSON
    response = requests.get(query_url).json()
    #print(response)
                
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in response["response"]["docs"]:
            #print(review)
            nyt_movies_list.append(review)
        
        # Print the page that was just retrieved
        print(f"Retrieving page {page} of results...")
    except:
        # Print the page number that had no results then break from the loop
        print(f"No results for page: {page}")
        break


Retrieving page 0 of results...
Retrieving page 1 of results...
Retrieving page 2 of results...
Retrieving page 3 of results...
Retrieving page 4 of results...
Retrieving page 5 of results...
Retrieving page 6 of results...
Retrieving page 7 of results...
Retrieving page 8 of results...
Retrieving page 9 of results...
Retrieving page 10 of results...
Retrieving page 11 of results...
Retrieving page 12 of results...
Retrieving page 13 of results...
Retrieving page 14 of results...
Retrieving page 15 of results...
Retrieving page 16 of results...
Retrieving page 17 of results...
Retrieving page 18 of results...
Retrieving page 19 of results...


In [8]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
#nyt_movies_list[:5]
print(json.dumps(nyt_movies_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [65]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_movies_df = pd.json_normalize(nyt_movies_list)
nyt_movies_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [66]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
nyt_movies_df["title"] = (nyt_movies_df["headline.main"].str.extract(r"(\u2018.*\u2019)"))

nyt_movies_df['title'] = nyt_movies_df['title'].astype("string")
nyt_movies_df['title'] = nyt_movies_df['title'].str.replace('\u2018', '')
nyt_movies_df['title'] = nyt_movies_df['title'].str.replace('\u2019', '')

nyt_movies_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Whats Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourists Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other Peoples Children


In [67]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
#nyt_movies_df["keywords"] = nyt_movies_df["keywords"].apply(extract_keywords)

clean_nyt_movies_df = nyt_movies_df[nyt_movies_df['title'].str.contains('<NA>') == False]

clean_nyt_movies_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Whats Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourists Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other Peoples Children


In [141]:
nyt_movies_df.dtypes

web_url                            object
snippet                            object
source                             object
keywords                           object
pub_date                           object
word_count                          int64
headline.main                      object
headline.kicker                    object
headline.content_kicker            object
headline.print_headline            object
headline.name                      object
headline.seo                       object
headline.sub                       object
byline.original                    object
byline.person                      object
byline.organization                object
title                      string[python]
dtype: object

In [69]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
movie_titles_list = clean_nyt_movies_df["title"].to_list()

movie_titles_list


['The Attachment Diaries',
 'Whats Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourists Guide to Love',
 'Other Peoples Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterleys Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate Review: Love the One You',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder Review: A God',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction Review: A Poet',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui

### Access The Movie Database API

In [70]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [71]:
# Create an empty list to store the results
tmdb_movies_list = []


# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0


# Loop through the titles
for title in movie_titles_list:

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        print("Sleeping...")
        time.sleep(10)


    # Add 1 to the request counter
    request_counter += 1

    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string).json()


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:

        # Get movie id
        movie_id = response["results"][0]["id"]


        # Make a request for a the full movie details


        # Execute "GET" request with url
        movie_response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}&language=en-US").json()

        #print(json.dumps(movie_response, indent=4))
        
        # Extract the genre names into a list
        genres = []
        for genre in movie_response["genres"]:
            genres.append(genre["name"])


        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for language in movie_response["spoken_languages"]:
            spoken_languages.append(language["english_name"])


        # Extract the production_countries' name into a list
        production_countries = []
        for country in movie_response["production_countries"]:
            production_countries.append(country["name"])


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({"title": movie_response["title"],
                                "release_date": movie_response["release_date"],
                                "budget": movie_response["budget"],
                                "revenue": movie_response["revenue"],
                                "runtime": movie_response["runtime"],
                                "genres": genres,
                                "spoken_languages": spoken_languages,
                                "production_countries": production_countries,
                                "vote_average": movie_response["vote_average"],
                                "vote_count": movie_response["vote_count"]})

        
        # Print out the title that was found
        print(f"Processing record {request_counter} | {title}")
    except:
        # Print out the title that was not found
        print(f"Movie not found. Skipping... {title}")



Processing record 1 | The Attachment Diaries
Processing record 2 | Whats Love Got to Do With It?
Processing record 3 | You Can Live Forever
Processing record 4 | A Tourists Guide to Love
Processing record 5 | Other Peoples Children
Processing record 6 | One True Loves
Processing record 7 | The Lost Weekend: A Love Story
Processing record 8 | A Thousand and One
Processing record 9 | Your Place or Mine
Processing record 10 | Love in the Time of Fentanyl
Processing record 11 | Pamela, a Love Story
Processing record 12 | In From the Side
Processing record 13 | After Love
Processing record 14 | Alcarràs
Processing record 15 | Nelly & Nadine
Processing record 16 | Lady Chatterleys Lover
Processing record 17 | The Sound of Christmas
Processing record 18 | The Inspection
Processing record 19 | Bones and All
Processing record 20 | My Policeman
Movie not found. Skipping... About Fate Review: Love the One You
Processing record 22 | Waiting for Bojangles
Processing record 23 | I Love My Dad
Proces

In [72]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))


[
    {
        "title": "The Attachment Diaries",
        "release_date": "2021-10-07",
        "budget": 0,
        "revenue": 0,
        "runtime": 102,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ],
        "vote_average": 4.0,
        "vote_count": 2
    },
    {
        "title": "What's Love Got to Do with It?",
        "release_date": "2023-01-26",
        "budget": 0,
        "revenue": 10898395,
        "runtime": 109,
        "genres": [
            "Romance",
            "Comedy"
        ],
        "spoken_languages": [
            "English",
            "Urdu",
            "Portuguese"
        ],
        "production_countries": [
            "United Kingdom"
        ],
        "vote_average": 6.0,
        "vote_count": 135
    },
    {
        "title": "You Can Live Foreve

In [73]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df.head()


,title,release_date,budget,revenue,runtime,genres,spoken_languages,production_countries,vote_average,vote_count
0,The Attachment Diaries,2021-10-07,0,0,102,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],4.000,2
1,What's Love Got to Do with It?,2023-01-26,0,10898395,109,"[Romance, Comedy]","[English, Urdu, Portuguese]",[United Kingdom],6.000,135
2,You Can Live Forever,2022-06-11,0,0,96,"[Drama, Romance]","[French, English]",[Canada],7.300,38
3,A Tourist's Guide to Love,2023-04-21,0,0,96,"[Romance, Comedy, Adventure]",[English],[United States of America],6.392,139
4,Other People's Children,2022-09-21,0,84178,104,"[Drama, Comedy]","[French, English]",[France],6.900,157


### Merge and Clean the Data for Export

In [120]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_movie_df = pd.merge(clean_nyt_movies_df, tmdb_movies_df, on="title", how="inner")

merged_movie_df.dropna()

#merged_movie_df.columns

merged_movie_df[['title','release_date','budget','runtime', 'genres','spoken_languages', 'production_countries', 'vote_average',
       'vote_count','byline.original']].head()


,title,release_date,budget,runtime,genres,spoken_languages,production_countries,vote_average,vote_count,byline.original
0,The Attachment Diaries,2021-10-07,0,102,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],4.000,2,By Jeannette Catsoulis
1,You Can Live Forever,2022-06-11,0,96,"[Drama, Romance]","[French, English]",[Canada],7.300,38,By Elisabeth Vincentelli
2,One True Loves,2023-04-07,0,100,"[Romance, Drama, Comedy]",[English],"[Germany, United States of America]",6.813,48,By Brandon Yu
3,The Lost Weekend: A Love Story,2023-04-13,0,95,[Documentary],[English],[United States of America],6.000,2,By Glenn Kenny
4,A Thousand and One,2023-03-31,0,116,[Drama],[English],[United States of America],7.000,50,By Manohla Dargis


In [121]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres", "spoken_languages", "production_countries", "keywords"]


# Create a list of characters to remove
characters_to_remove = ["[", "]", "'"]


# Loop through the list of columns to fix
for column in columns_to_fix:

    # Convert the column to type 'str'
    merged_movie_df[column] = merged_movie_df[column].astype("str")


    # Loop through characters to remove
    for character in characters_to_remove:

        # Remove the character
        merged_movie_df[column] = merged_movie_df[column].str.replace(character, "")


# Display the fixed DataFrame
merged_movie_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,title,release_date,budget,revenue,runtime,genres,spoken_languages,production_countries,vote_average,vote_count
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"{name: subject, value: Movies, rank: 1, major:...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,The Attachment Diaries,2021-10-07,0,0,102,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,4.000,2
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"{name: subject, value: Movies, rank: 1, major:...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,You Can Live Forever,2022-06-11,0,0,96,"Drama, Romance","French, English",Canada,7.300,38
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"{name: subject, value: Movies, rank: 1, major:...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,One True Loves,2023-04-07,0,37820,100,"Romance, Drama, Comedy",English,"Germany, United States of America",6.813,48
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"{name: subject, value: Documentary Films and P...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,The Lost Weekend: A Love Story,2023-04-13,0,0,95,Documentary,English,United States of America,6.000,2
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"{name: subject, value: Movies, rank: 1, major:...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,A Thousand and One,2023-03-31,0,0,116,Drama,English,United States of America,7.000,50


In [122]:
# Drop "byline.person" column
merged_movie_df.drop(columns=["byline.person"], inplace=True)
merged_movie_df[['title','release_date','budget','runtime', 'genres','spoken_languages', 'production_countries', 'vote_average',
       'vote_count','byline.original']].head()


,title,release_date,budget,runtime,genres,spoken_languages,production_countries,vote_average,vote_count,byline.original
0,The Attachment Diaries,2021-10-07,0,102,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,4.000,2,By Jeannette Catsoulis
1,You Can Live Forever,2022-06-11,0,96,"Drama, Romance","French, English",Canada,7.300,38,By Elisabeth Vincentelli
2,One True Loves,2023-04-07,0,100,"Romance, Drama, Comedy",English,"Germany, United States of America",6.813,48,By Brandon Yu
3,The Lost Weekend: A Love Story,2023-04-13,0,95,Documentary,English,United States of America,6.000,2,By Glenn Kenny
4,A Thousand and One,2023-03-31,0,116,Drama,English,United States of America,7.000,50,By Manohla Dargis


In [125]:
# Delete duplicate rows and reset index
#merged_movie_df.dtypes

merged_movie_df.drop_duplicates(inplace=True)
merged_movie_df.reset_index(drop=True, inplace=True)

merged_movie_df[['title','release_date','budget','runtime', 'genres','spoken_languages', 'production_countries', 'vote_average','vote_count','byline.original']].head()


,title,release_date,budget,runtime,genres,spoken_languages,production_countries,vote_average,vote_count,byline.original
0,The Attachment Diaries,2021-10-07,0,102,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,4.000,2,By Jeannette Catsoulis
1,You Can Live Forever,2022-06-11,0,96,"Drama, Romance","French, English",Canada,7.300,38,By Elisabeth Vincentelli
2,One True Loves,2023-04-07,0,100,"Romance, Drama, Comedy",English,"Germany, United States of America",6.813,48,By Brandon Yu
3,The Lost Weekend: A Love Story,2023-04-13,0,95,Documentary,English,United States of America,6.000,2,By Glenn Kenny
4,A Thousand and One,2023-03-31,0,116,Drama,English,United States of America,7.000,50,By Manohla Dargis


In [128]:
# Export data to CSV without the index
merged_movie_df.to_csv("output/nyt_movie_data.csv", index=False, header=True)
